In [1]:
import time
import pandas as pd

start = time.time()
service_labels = pd.read_csv('../data/url_domain_1to1_mappings.csv')
news_labels = pd.read_csv('https://raw.githubusercontent.com/LazerLab/DomainDemo/refs/heads/main/data/existing_labels/news_local_natioanl_classification.csv')
print(f"Loaded datasets in {(time.time() - start):2f} seconds")

Loaded datasets in 1.073114 seconds


In [2]:
service_labels.columns = ['domain','label']
service_labels['domain'] = service_labels.domain.str.replace('www.','')
service_labels.sample(5)

,domain,label
3697,freebeacon.com,News
3270,dearauthor.com,Social Media/Forums
1793,electricoceanband.com,Entertainment & Culture
3482,therealnews.com,News
829,jeb.biologists.org,"Science, Academia, & Technology"


In [3]:
combined_labels = service_labels.copy()
combined_labels['label_source'] = 'data_provenance_init'

news_domains = set(news_labels['domain'])
combined_labels.loc[combined_labels['domain'].isin(news_domains), 'label'] = "News"
combined_labels.loc[combined_labels['domain'].isin(news_domains), 'label_source'] = "northeastern_domain_demo"

# Add missing domains from news_labels
missing_domains = news_labels[~news_labels['domain'].isin(combined_labels['domain'])]
if not missing_domains.empty:
    new_rows = pd.DataFrame({
        'domain': missing_domains['domain'],
        'label': "News",
        'label_source': "northeastern_domain_demo"
    })
    combined_labels = pd.concat([combined_labels, new_rows], ignore_index=True)

In [4]:
combined_labels.sample(10)

,domain,label,label_source
4608,crisismagazine.com,News,northeastern_domain_demo
14497,grantcountynewspapers.com,News,northeastern_domain_demo
13889,newpaltzx.com,News,northeastern_domain_demo
5946,arizonaguide.com,News,northeastern_domain_demo
2622,vogue.com,News,data_provenance_init
7256,greenepublishing.com,News,northeastern_domain_demo
4100,pillohotelashbourne.com,Business & E-Commerce,data_provenance_init
13054,sitraders.com,News,northeastern_domain_demo
7590,kakm.org,News,northeastern_domain_demo
15139,altonadvertiser.com,News,northeastern_domain_demo


In [5]:
combined_labels.label.value_counts()

label
News                               13893
Business & E-Commerce                780
Entertainment & Culture              597
Science, Academia, & Technology      471
General Information & Education      426
Social Media/Forums                  151
Legal & Policy                       136
Blogs                                135
Other                                109
Books                                 58
Name: count, dtype: int64

In [6]:
len(combined_labels)

16759

In [7]:
combined_labels.to_csv('../data/combined_domain_labels_16k.csv',index=False)